# 03 - Sliding Window Analysis
## Detecting Cut-off Bluetooth Signals

This notebook covers:
- Temporal sliding window implementation
- Bluetooth signal cutoff detection
- Visualization of detections
- Analysis of frequency continuity

In [ ]:
# Setup
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2

from src.slicing import sliding_window_bluetooth_detection
from src.visualization import visualize_cutoff_detections

print("✅ All imports successful!")

## Configuration

In [ ]:
# Paths
SPECTROGRAM_DIR = Path('../data/spectrograms')
RESULTS_DIR = Path('../results/cutoff_detections')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Sliding window parameters
WINDOW_SIZE = 5  # Number of consecutive frames
STRIDE = 1  # Step size

print(f"Window size: {WINDOW_SIZE} frames")
print(f"Stride: {STRIDE} frame(s)")

## Run Sliding Window Detection

In [ ]:
# Detect Bluetooth cutoffs
detections = sliding_window_bluetooth_detection(
    spectrogram_dir=SPECTROGRAM_DIR,
    window_size=WINDOW_SIZE,
    stride=STRIDE
)

print(f"Found {len(detections)} potential Bluetooth cutoff events")

# Display detection summary
if len(detections) > 0:
    print("\nTop 10 detections by confidence:")
    sorted_detections = sorted(detections, 
                              key=lambda x: x['cutoff_info']['confidence'], 
                              reverse=True)
    
    for idx, det in enumerate(sorted_detections[:10], 1):
        conf = det['cutoff_info']['confidence']
        print(f"{idx}. Frames {det['start_frame']}-{det['end_frame']}: confidence={conf:.3f}")

## Visualize Detections

In [ ]:
# Generate visualization images
if len(detections) > 0:
    visualize_cutoff_detections(
        spectrogram_dir=SPECTROGRAM_DIR,
        detections=detections,
        output_dir=RESULTS_DIR
    )
    print(f"Visualizations saved to {RESULTS_DIR}")

## Analysis: Cutoff Statistics

In [ ]:
if len(detections) > 0:
    confidences = [d['cutoff_info']['confidence'] for d in detections]
    
    plt.figure(figsize=(10, 6))
    plt.hist(confidences, bins=20, edgecolor='black')
    plt.xlabel('Confidence Score')
    plt.ylabel('Count')
    plt.title('Distribution of Bluetooth Cutoff Confidence Scores')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"Mean confidence: {np.mean(confidences):.3f}")
    print(f"Median confidence: {np.median(confidences):.3f}")
    print(f"High confidence (>0.5): {sum(c > 0.5 for c in confidences)} detections")

## Next Steps

Continue to notebook 04 for signal detection and classification.